#### [Pytorch 기반 회귀 모델 만들기]
- layer = Full - Connected layer, linear
- 출력값 : 1개. 
- 손실함수 : MSELoss, MAELoss

[ 1 ] data 준비 <hr>

- 모듈 로딩

In [2]:
import pandas as pd
import torch

In [3]:
data = pd.read_csv('../../datas/BostonHousing.csv')
data.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [18]:
import numpy as np

In [8]:
target = pd.DataFrame(data['medv'])
features = data.drop('medv', axis=1)

In [7]:
target.shape, features.shape

((506,), (506, 13))

In [ ]:
target.info ,features.info

In [13]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(features, target)

[ 2 ] 사용자 데이터셋 클래스 정의하기 <hr>

In [30]:
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

In [16]:
class BostonDS(Dataset):
    def __init__(self, feature, target):
        super().__init__()
        scaler = MinMaxScaler()
        self.feature = torch.FloatTensor(scaler.fit_transform(feature))
        self.target = torch.FloatTensor(target)
        self.len = self.feature.shape[0]

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.feature[idx], self.target[idx]
    
    # 데이터 정규화 기능 함수
    def normalization(self):
        self.feature.max(dim=1)



## __어쩌구__함수는 무조건 필수


In [19]:
xtrain = xtrain.to_numpy()
ytrain = ytrain.to_numpy()
xtest = xtest.to_numpy()
ytest = ytest.to_numpy()

train_ds = BostonDS(xtrain, ytrain)
test_ds = BostonDS(xtest, ytest)

In [20]:
# 모델 정의하기
import torch.nn as nn

class BostonRegression(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
    

In [21]:
model = BostonRegression(input_size=13, hidden_size=10, output_size=1)

In [32]:
import torch.optim as optim

batch_size = 32
trainDL = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
testDL = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)
num_epochs = 10000

# 학습 및 평가 과정에서 테스트 데이터 손실 계산
for ep in range(num_epochs):
    model.train()
    for inputs, targets in trainDL:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for inputs, targets in testDL:
            outputs = model(inputs)
            loss = criterion(outputs, targets.unsqueeze(1))
            test_loss += loss.item()
        test_loss /= len(testDL)
    print(f'[EP] {ep+1}/{num_epochs}\n [LOSS]{test_loss:.2f}')

[EP] 1/10000
 [LOSS]99.44
[EP] 2/10000
 [LOSS]100.21
[EP] 3/10000
 [LOSS]99.80
[EP] 4/10000
 [LOSS]99.96
[EP] 5/10000
 [LOSS]99.61
[EP] 6/10000
 [LOSS]99.66
[EP] 7/10000
 [LOSS]99.75
[EP] 8/10000
 [LOSS]99.70
[EP] 9/10000
 [LOSS]99.92
[EP] 10/10000
 [LOSS]99.76
[EP] 11/10000
 [LOSS]99.80
[EP] 12/10000
 [LOSS]99.74
[EP] 13/10000
 [LOSS]99.98
[EP] 14/10000
 [LOSS]99.71
[EP] 15/10000
 [LOSS]99.75
[EP] 16/10000
 [LOSS]99.45
[EP] 17/10000
 [LOSS]99.83
[EP] 18/10000
 [LOSS]100.01
[EP] 19/10000
 [LOSS]99.77
[EP] 20/10000
 [LOSS]99.85
[EP] 21/10000
 [LOSS]99.93
[EP] 22/10000
 [LOSS]99.68
[EP] 23/10000
 [LOSS]99.62
[EP] 24/10000
 [LOSS]100.01
[EP] 25/10000
 [LOSS]99.63
[EP] 26/10000
 [LOSS]99.77
[EP] 27/10000
 [LOSS]99.51
[EP] 28/10000
 [LOSS]99.98
[EP] 29/10000
 [LOSS]100.09
[EP] 30/10000
 [LOSS]99.63
[EP] 31/10000
 [LOSS]99.60
[EP] 32/10000
 [LOSS]99.81
[EP] 33/10000
 [LOSS]99.70
[EP] 34/10000
 [LOSS]99.70
[EP] 35/10000
 [LOSS]99.53
[EP] 36/10000
 [LOSS]100.01
[EP] 37/10000
 [LOSS]99.94
[EP] 

[EP] 100/100
 [LOSS]0.00


c:\Users\kdp\anaconda3\envs\NLP_38\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\kdp\anaconda3\envs\NLP_38\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([31, 1, 1])) that is different to the input size (torch.Size([31, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
